# PPO Breakout Example

### Atari Breakout

This example adds a temporal aspect to the game of breakout by feeding 3 frames of input at a time as opposed to just one as done in the "ppo_breakout" example. While this is not the only way to add temporal relevance (LSTMs are a popular alternative/addition), it is simple and demonstrates how to fully make use of the library by making use of more in depth parameters and modifying a memory buffer.

Please do note that this example may take a long time to train.

With the default 4 threads runnning on an 8-core CPU with a GTX 1080 Ti, it will take several hours to train to a decent level of play.

Running on a platform with more GPU power and a larger cluster of CPUs could siginificantly reduce training time.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten
import gym
from ludus.policies import PPOTrainer
from ludus.env import EnvController
from ludus.utils import preprocess_atari
from ludus.memory import MTMemoryBuffer
import copy

In [2]:
env = gym.make('Breakout-v0') # This instance of the environment is only used
                              # to get action dimensions
in_shape = [84, 84, 3] # Size of reshaped observations

# Creating a conv net for the policy and value estimator
obs_op = Input(shape=in_shape)
conv1 = Conv2D(16, 8, (4, 4), activation='relu')(obs_op)
max_pool1 = MaxPool2D(2, 2)(conv1)
conv2 = Conv2D(32, 4, (2, 2), activation='relu')(max_pool1)
max_pool2 = MaxPool2D(2, 2)(conv2)
dense1 = Dense(256, activation='relu')(max_pool2)
flattened = Flatten()(dense1)

# Output probability distribution over possible actions
act_probs_op = Dense(env.action_space.n, activation='softmax')(flattened)

# Output value of observed state
value_op = Dense(1)(flattened)

# Wrap a Proximal Policy Optimization Trainer on top of the network
network = PPOTrainer(obs_op, act_probs_op, value_op, act_type='discrete', ppo_iters=80)

/home/ejmejm/anaconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [3]:
n_episodes = 10000 # Total episodes of data to collect
max_steps = 2048 # Max number of frames per game
batch_size = 12 # Smaller = faster, larger = stabler
print_freq = 10 # How many training updates between printing progress

In [4]:
agent_hist = {} # Keeps track of up to 3 previous frames for each agent

# Create observation transformation that adds the two last frames on
# as two extra dimensions
def new_obs_transform(obs, agent_id):
    new_frame = preprocess_atari(obs.squeeze()) # First preprocess the new frame
    
    if agent_id in agent_hist: # Case for a continued episode
        agent_hist[agent_id] = agent_hist[agent_id][1:]
        agent_hist[agent_id].append(new_frame)
    else: # Case for a new episode
        agent_hist[agent_id] = [new_frame, new_frame, new_frame]
    
    # Format the data
    arr = np.array(agent_hist[agent_id])
    return np.swapaxes(arr, 0, 3).squeeze()

############################################################
############################################################

mtmb = MTMemoryBuffer() # Create a memory buffer to store the episode data

# Edit the memory buffer's start_rollout function so that every time
# an episode ends, it resets the respective agent's history
old_start_rollout = mtmb.start_rollout

def new_start_rollout(agent_id):
    old_start_rollout(agent_id)
    agent_hist.pop(agent_id, None)
    
mtmb.start_rollout = new_start_rollout

In [5]:
# Create the environment controller for generating game data
ec = EnvController(lambda: gym.make('Breakout-v0'), n_threads=4, memory_buffer=mtmb)
# Set the preprocessing function for observations
ec.set_obs_transform(new_obs_transform)

In [ ]:
update_rewards = []

for i in range(int(n_episodes / batch_size)):
    ec.sim_episodes(network, batch_size, max_steps) # Simualate env to generate data
    update_rewards.append(ec.get_avg_reward()) # Append rewards to reward tracker list
    dat = ec.get_data() # Get all the data gathered
    network.train(dat) # Train the network with PPO
    if i != 0 and i % print_freq == 0:
        print(f'Update #{i}, Avg Reward: {np.mean(update_rewards[-print_freq:])}') # Print an update

/home/ejmejm/anaconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


Update #10, Avg Reward: 13.558333333333332
Update #20, Avg Reward: 13.216666666666669


In [8]:
ec.render_episodes(network, 5, max_steps) # Render an episode to see the result

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ejmejm/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3265, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-3eb16c1aba83>", line 1, in <module>
    ec.render_episodes(network, 5, max_steps) # Render an episode to see the result
  File "/home/ejmejm/MLProjects/ludus/env.py", line 191, in render_episodes
    time.sleep(0.02)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ejmejm/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2016, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ejmejm/anaconda3/lib/python3.6/site-packages/IPython/core/ultratb.py",

KeyboardInterrupt: 

In [15]:
saver = tf.train.Saver()
saver.save(network.sess, 'models/breakout_temporal.ckpt')

'models/breakout_temporal.ckpt'